In [1]:
from cfg.configfile_req import *
from lib.csysrq_support import *
from redminelib import Redmine

redmine = Redmine(req_server_url,key=req_key_txt)
projects = redmine.project.all()

print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)
    

Proyectos:
	 reqtest  	|  Requirements test


In [2]:
my_project = redmine.project.get(req_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

Obtenemos proyecto:  reqtest  |  Requirements test


In [3]:
tmp = redmine.issue.filter(project_id=req_project_id_str, tracker_id=req_rq_tracker_id)
my_project_issues = sorted(tmp, key=lambda k: k.subject)
tmp = redmine.issue.filter(project_id=req_project_id_str, tracker_id=req_doc_tracker_id)
my_doc_issues = sorted(tmp, key=lambda k: k.subject)

Conectaremos con nuestra instancia de PYOO
https://github.com/seznam/pyoo

In [4]:
import pyoo
desktop = pyoo.Desktop('localhost', 2002)

Copiamos el template del fichero de exportación al nombre de exportación

In [5]:
from shutil import copyfile

copyfile('./downloading/templates/RqDownload.ods', './downloading/RqDownload.ods')

'./downloading/RqDownload.ods'

Hemos de cargar los RqTarget

In [6]:
# Conectamos con la hoja
doc = desktop.open_spreadsheet('./downloading/RqDownload.ods')

# La lista de RqTarget empieza en B7, hacia abajo
rq_target_column = 2
rq_target_row = 8



In [9]:
print(dir(doc))
print(len(doc.sheets))
doc_dict = doc.sheets['Dict']
print(doc_dict)
#print(doc_dict[rq_target_row,rq_target_column].address)
#doc_dict[rq_target_row,rq_target_column].value = 5

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_null_date', '_sheets', '_target', 'close', 'date_from_number', 'date_to_number', 'get_locale', 'save', 'sheets', 'time_from_number', 'time_to_number']
3


com.sun.star.uno.RuntimeException: Binary URP bridge disposed during call

Ahora generaremos los documentos a partir de aquellos requisitos que tengan algún hijo

In [5]:
from doorstop.core.item import Item

tree = doorstop.build()

for my_issue in my_doc_issues:
    print("********** ",my_issue)
    docpath = "./doorstop/"+my_issue.subject
    prefix = my_issue.subject+"-"
    current_parent = getattr(my_issue, 'parent', None)
    if current_parent is None:
        doc = tree.create_document(docpath,prefix)
        parent_issue = None
    else:
        parent_issue = redmine.issue.get(current_parent.id)
        print("parent: ",parent_issue.subject)
        doc = tree.create_document(docpath,prefix,parent=parent_issue.subject+"-")
       
    print("prefix: ",prefix)
    reqname = my_issue.subject
    print("reqname: ",reqname)
    current_version = getattr(my_issue, 'fixed_version', None)
    if (current_version is not None):
        print("target:",current_version)
        print(dir(current_version))
        version_name = current_version.name
    else:
        version_name = ''

    doc.set('BDID', str(my_issue.id))
    doc.set('BDURL', req_server_url+"/issues/"+str(my_issue.id))
    doc.set('RqSubject', my_issue.subject)
    doc.set('RqTitle', my_issue.custom_fields.get(req_title_cf_id).value)
    if (parent_issue is not None):
        doc.set('RqParent', parent_issue.subject)
    else:
        doc.set('RqParent', '')
    doc.set('RqRationale', my_issue.custom_fields.get(req_rationale_cf_id).value)
    doc.set('RqLevel', my_issue.custom_fields.get(req_level_cf_id).value)
    doc.set('RqType', my_issue.custom_fields.get(req_type_cf_id).value)
    doc.set('RqSources', my_issue.custom_fields.get(req_sources_cf_id).value)
    doc.set('RqChapter', my_issue.custom_fields.get(req_chapter_cf_id).value)
    doc.set('RqTarget', version_name)
    doc.set('text', my_issue.description)
    
    # Ahora grabamos un requisito cero:
    newitem = Item.new(tree, doc,
        doc.path, doc.root, my_issue.subject+"-0000",
        auto=False)
    newitem.set('BDID', str(my_issue.id))
    newitem.set('BDURL', req_server_url+"/issues/"+str(my_issue.id))
    newitem.set('RqSubject', my_issue.subject)
    newitem.set('RqTitle', my_issue.custom_fields.get(req_title_cf_id).value)
    if (parent_issue is not None):
        newitem.set('RqParent', parent_issue.subject)
        
    else:
        newitem.set('RqParent', '')
        
    newitem.set('RqRationale', my_issue.custom_fields.get(req_rationale_cf_id).value)
    newitem.set('RqLevel', my_issue.custom_fields.get(req_level_cf_id).value)
    newitem.set('RqType', my_issue.custom_fields.get(req_type_cf_id).value)
    newitem.set('RqSources', my_issue.custom_fields.get(req_sources_cf_id).value)
    newitem.set('RqChapter', my_issue.custom_fields.get(req_chapter_cf_id).value)
    newitem.set('RqTarget', version_name)
    newitem.set('text', my_issue.description)
    newitem.save()

**********  SR
prefix:  SR-
reqname:  SR
target: v0.1
['id', 'name']
**********  SRC
parent:  SR
prefix:  SRC-
reqname:  SRC


Ahora crearemos los requisitos "hijos" dentro de cada documento

In [6]:
def find_prefix(this_issue):
    if this_issue.tracker.id == req_doc_tracker_id:
        return(this_issue.subject+"-")

    # not do found yet
    current_parent = getattr(this_issue, 'parent', None)
    if current_parent is None:
        return ""
    
    else:
        parent_issue = redmine.issue.get(current_parent.id)
        print("Llamo al padre")
        return find_prefix(parent_issue)
    

from doorstop.core.item import Item

for my_issue in my_project_issues:
    reqname = my_issue.subject
    print("reqname: ",reqname)
    current_parent = getattr(my_issue, 'parent', None)
    if current_parent is None:
        prefix = ""
        
    else:    
        parent_issue = redmine.issue.get(my_issue.parent.id)
        print("parent: ",parent_issue.subject)
        prefix = find_prefix(parent_issue)
    
    print(dir(my_issue))
    current_version = getattr(my_issue, 'fixed_version', None)
    if (current_version is not None):
        print("target:",current_version)
        print(dir(current_version))
        version_name = current_version.name
    else:
        version_name = ''
    if (prefix is not None):
        print("prefix: ",prefix)        
        #newitem = tree.add_item(prefix)
        #newitem.text = my_issue.description

        document = tree.find_document(prefix)
        newitem = Item.new(tree, document,
                document.path, document.root, my_issue.subject,
                auto=False)
        newitem.set('BDID', str(my_issue.id))
        newitem.set('BDURL', req_server_url+"/issues/"+str(my_issue.id))
        newitem.set('RqSubject', my_issue.subject)
        newitem.set('RqTitle', my_issue.custom_fields.get(req_title_cf_id).value)
        if (parent_issue is not None):
            newitem.set('RqParent', parent_issue.subject)
        
        else:
            newitem.set('RqParent', '')

        newitem.set('RqRationale', my_issue.custom_fields.get(req_rationale_cf_id).value)
        newitem.set('RqLevel', my_issue.custom_fields.get(req_level_cf_id).value)
        newitem.set('RqType', my_issue.custom_fields.get(req_type_cf_id).value)
        newitem.set('RqSources', my_issue.custom_fields.get(req_sources_cf_id).value)
        newitem.set('RqChapter', my_issue.custom_fields.get(req_chapter_cf_id).value)
        newitem.set('RqTarget', version_name)
        newitem.set('text', my_issue.description)
        newitem.save()
        print("---->",newitem)


reqname:  SR-0001
parent:  SR
['attachments', 'author', 'changesets', 'children', 'created_on', 'custom_fields', 'description', 'done_ratio', 'id', 'journals', 'parent', 'priority', 'project', 'relations', 'status', 'subject', 'time_entries', 'tracker', 'updated_on', 'watchers']
prefix:  SR-
----> SR-0001
reqname:  SR-0002
parent:  SR-0001
Llamo al padre
['attachments', 'author', 'changesets', 'children', 'created_on', 'custom_fields', 'description', 'done_ratio', 'fixed_version', 'id', 'journals', 'parent', 'priority', 'project', 'relations', 'status', 'subject', 'time_entries', 'tracker', 'updated_on', 'watchers']
target: v0.1
['id', 'name']
prefix:  SR-
----> SR-0002
reqname:  SR-0003
parent:  SR
['attachments', 'author', 'changesets', 'children', 'created_on', 'custom_fields', 'description', 'done_ratio', 'id', 'journals', 'parent', 'priority', 'project', 'relations', 'status', 'subject', 'time_entries', 'tracker', 'updated_on', 'watchers']
prefix:  SR-
----> SR-0003
reqname:  SR-0

In [7]:
# Creamos los links entre requisitos del Redmine
for i in my_project_issues:
    thisitem = tree.find_item(i.subject)
    if (thisitem is not None):
        print(" thisitem: ",thisitem)

    # Busco las relaciones en las que es destinatario
    my_issue_relations = redmine.issue_relation.filter(issue_id=i.id)
    my_filtered_issue_relations = list(filter(lambda x: x.issue_id != i.id, my_issue_relations))

    # Recorro las relaciones creando los links
    for rel in my_filtered_issue_relations:
        # Obtenemos la incidencia y el item doorstop del objeto que es origen de la relación de Redmine,
        # que significa que es destinatario de la relación de doorstop, ya que es el elemento que está
        # condicionando al actual (el actual depende de él)
        relissue = redmine.issue.get(rel.issue_id)
        print("Relacionado: ",rel," de ",relissue.subject," a ",i.subject)
        relitem = tree.find_item(relissue.subject)
        # Creamos un link donde expresamos que thisitem depende de relitem
        thisitem.link(relitem)

    
print("Acabamos")


 thisitem:  SR-0001
 thisitem:  SR-0002
 thisitem:  SR-0003
 thisitem:  SR-0004
 thisitem:  SR-0005
 thisitem:  SR-0006
 thisitem:  SR-0007
Relacionado:  8  de  SR-0005  a  SR-0007
Relacionado:  9  de  SR-0006  a  SR-0007
 thisitem:  SR-0008
Relacionado:  10  de  SR-0005  a  SR-0008
 thisitem:  SR-0009
 thisitem:  SR-0010
 thisitem:  SRC-0001
 thisitem:  SRC-0002
 thisitem:  SRC-0003
 thisitem:  SRC-0004
 thisitem:  SRC-0005
Relacionado:  11  de  SR-0004  a  SRC-0005
 thisitem:  SRC-0006
 thisitem:  SRC-0007
Relacionado:  12  de  SRC-0005  a  SRC-0007
Relacionado:  13  de  SRC-0006  a  SRC-0007
 thisitem:  SRC-0008
Relacionado:  14  de  SRC-0005  a  SRC-0008
 thisitem:  SRC-0009
 thisitem:  SRC-0010
Acabamos
